# Phase 4: Conversation Memory and Polish

In this final notebook, you'll learn how to:
1. Add conversation memory for follow-up questions
2. Include source citations in responses
3. Build a conversational RAG chain

In [ ]:
import sys
sys.path.insert(0, '../..')

## 1. Simple Conversation Memory

Conversation memory keeps track of previous exchanges to enable follow-up questions.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Simple list-based memory
chat_history = []

def add_to_history(human_msg: str, ai_msg: str):
    """Add a conversation turn to history."""
    chat_history.append(HumanMessage(content=human_msg))
    chat_history.append(AIMessage(content=ai_msg))

# Simulate a conversation
add_to_history("What is machine learning?", "Machine learning is a type of AI...")
add_to_history("What are the types?", "There are three main types: supervised, unsupervised, and reinforcement learning.")

print("Chat history:")
for msg in chat_history:
    role = "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content[:50]}...")

## 2. RAG with Conversation History

We can modify our RAG chain to include conversation history for context.

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# Create a prompt that includes chat history
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that answers questions based on the provided context.
Use the context and conversation history to provide accurate answers.

Context:
{context}"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

print("Conversational prompt template created!")
print(conversational_prompt.input_variables)

In [ ]:
from src.document_loader import load_and_split
from src.vectorstore import get_or_create_vectorstore, get_retriever

# Set up the RAG components
chunks = load_and_split()
vectorstore = get_or_create_vectorstore(chunks)
retriever = get_retriever(vectorstore)
llm = ChatOllama(model="llama3.2:3b")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Create a conversational RAG function
def conversational_rag(question: str, chat_history: list) -> str:
    """RAG with conversation history."""
    # Retrieve relevant documents
    docs = retriever.invoke(question)
    context = format_docs(docs)
    
    # Create the messages
    messages = conversational_prompt.invoke({
        "context": context,
        "chat_history": chat_history,
        "question": question
    })
    
    # Get the response
    response = llm.invoke(messages)
    return response.content

In [ ]:
# Test the conversational RAG
history = []

# First question
q1 = "What is supervised learning?"
a1 = conversational_rag(q1, history)
print(f"Q: {q1}")
print(f"A: {a1}\n")

# Add to history
history.append(HumanMessage(content=q1))
history.append(AIMessage(content=a1))

# Follow-up question (uses "it" referring to supervised learning)
q2 = "What are some examples of it?"
a2 = conversational_rag(q2, history)
print(f"Q: {q2}")
print(f"A: {a2}")

## 3. Source Citations

It's useful to show users which documents were used to generate an answer.

In [ ]:
from src.rag_chain import create_rag_chain_with_sources

# Create RAG chain that returns sources
rag_with_sources = create_rag_chain_with_sources(vectorstore)

# Ask a question
result = rag_with_sources("What is reinforcement learning?")

print("Answer:")
print(result['answer'])
print("\n" + "="*50)
print("\nSources used:")
for i, doc in enumerate(result['sources'], 1):
    print(f"\n[{i}] {doc.page_content[:150]}...")

## 4. Using the CLI

We've created a CLI in `src/cli.py` that combines all these features. Run it from the terminal:

```bash
# Activate the virtual environment
source venv/bin/activate

# Run the CLI
python -m src.cli
```

The CLI supports:
- Interactive Q&A
- Viewing source documents
- Conversation history
- Reloading documents

## Congratulations! 🎉

You've completed the LangChain RAG learning path. You now understand:

1. **LangChain Basics** - Connecting to LLMs, prompts, and chains
2. **Document Processing** - Loading, splitting, and embedding documents
3. **Vector Stores** - Storing and searching embeddings with ChromaDB
4. **RAG Chains** - Combining retrieval with generation
5. **Conversation Memory** - Enabling follow-up questions
6. **Source Citations** - Showing document provenance

## Next Steps

To continue learning:

1. **Add more documents** - Try PDFs, different topics
2. **Experiment with prompts** - Customize the system prompt
3. **Try different models** - Swap llama3.2 for mistral or others
4. **Add a web UI** - Build with Streamlit or Gradio
5. **Explore agents** - LangChain agents can use tools and make decisions

Happy coding! 🚀